In [6]:
import torch
from torchsummaryX import summary
import torch 
import os
import numpy as np
import nibabel as nib
import imageio.v2 as im
import matplotlib.pyplot as plt
import glob
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torchvision import transforms, utils
import pdb
import datetime
from torchmetrics import JaccardIndex
from torch.utils.tensorboard import SummaryWriter
from ResUnet7TFLAIR import *
import tqdm
import gc
from natsort import natsorted
torch.cuda.get_device_name(0)

ModuleNotFoundError: No module named 'imageio'

In [5]:
!pip install nibabel

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 3.3/3.3 MB 15.3 MB/s eta 0:00:00


In [8]:
class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 1.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
            y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

def readImage(path,subjN):
    img_array = []
    img_dir = []
    affine_array = []
    counter = 0
    for directory_path in path:
        img_dir.append(directory_path)
    img_dir.sort()

    for path_i in img_dir:
        img = nib.load(path_i).get_fdata()
        img = img/np.max(img)
        affine = nib.load(path_i).affine
        img_array.append(img)
        affine_array.append(affine)
        counter += 1
        if counter >= subjN: break

    img_array = np.array(img_array,dtype=object)
    affine_array = np.array(affine_array,dtype=object)
    print(path_i)
    return img_array, affine_array
 
def sliceImage(image, output_size):
    image_stack = []
    new_h, new_w, new_z = output_size
    normalization=transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    for img in image:
        h, w, z = np.shape(img)
        if h < new_h:
            pad_size = int((new_h - h)/2)
            img = np.pad(img, ((pad_size, pad_size), (0,0), (0,0)))
        elif h > new_h:
            crop_size = int((h - new_h)/2)
            img = img[crop_size:h-crop_size, :, :]
        if w < new_w:
            pad_size = int((new_w - w)/2)
            img = np.pad(img, ((0,0), (pad_size, pad_size), (0,0)))
        elif w > new_w:
            crop_size = int((w - new_w)/2)
            img = img[:, crop_size:w-crop_size, :]
        pad_size = int(new_z/2)
        img = np.pad(img, ((0,0), (0,0), (pad_size, pad_size)))
        for z in range(np.shape(img)[2]):
            img_z = np.rot90(img[:,:,z:z+new_z])
            #img_z = np.expand_dims(img_z, 0)
            image_stack.append(img_z)
    return np.array(image_stack)

def load_train_val(train, val, size, num):
    img_train = glob.glob(train['img'])
    mask_train = glob.glob(train['mask'])
    len_train_mask=len(mask_train)
    
    
    img_test = glob.glob(val['img'])
    mask_test = glob.glob(val['mask'])
    len_test_mask=len(mask_test)
    
    img_train = natsorted(img_train)
    mask_train = natsorted(mask_train)
    #img_train=img_train[-len_train_mask:]
    
    img_test = natsorted(img_test)
    mask_test = natsorted(mask_test)
    
    #img_test = img_test[-len_train_mask:]
    FLAIR_train, FLAIR_train_affine = readImage(img_train, num)
    FLAIR_train_mask, _ = readImage(mask_train, num)
    FLAIR_train_stack = sliceImage(FLAIR_train, size).astype('float')
    mask_train_stack = sliceImage(FLAIR_train_mask, size).astype('float')
    
    
    FLAIR_test, FLAIR_test_affine = readImage(img_test, 5)
    FLAIR_test_mask, _ = readImage(mask_test, 5)
    FLAIR_test_stack = sliceImage(FLAIR_test, size).astype('float')
    mask_test_stack = sliceImage(FLAIR_test_mask, size).astype('float')
    
    flair_trainset = TensorDataset(torch.tensor(FLAIR_train_stack), torch.tensor(mask_train_stack))
    flair_testset = TensorDataset(torch.tensor(FLAIR_test_stack), torch.tensor(mask_test_stack))
    
    return flair_trainset, flair_testset
    


In [9]:
config = {
    'batch_size': 64, # Increase this if your GPU can handle it
    'lr': 0.0001,
    'epochs': 100, # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    # Include other parameters as needed.
}

criterion = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.8, patience=2)

In [10]:
natsorted(glob.glob('/ocean/projects/cis220078p/zli11/MRIPublic/public/dataset/*2/orig/FLAIR_mask.nii.gz'))


['/ocean/projects/cis220078p/zli11/MRIPublic/public/dataset/102/orig/FLAIR_mask.nii.gz',
 '/ocean/projects/cis220078p/zli11/MRIPublic/public/dataset/112/orig/FLAIR_mask.nii.gz',
 '/ocean/projects/cis220078p/zli11/MRIPublic/public/dataset/132/orig/FLAIR_mask.nii.gz']

In [11]:
path_list=natsorted(glob.glob('/ocean/projects/cis220078p/zli11/MRIPublic/public/dataset/*2/orig/FLAIR_mask.nii.gz'))
for path in path_list:
    print(nib.load(path).get_fdata().shape)

(132, 256, 83)
(132, 256, 83)
(132, 256, 83)


In [12]:
train = {'img': '/ocean/projects/cis220078p/zli11/MRIPublic/public/AMS/*/orig/FLAIR.nii.gz', 
        'mask': '/ocean/projects/cis220078p/zli11/MRIPublic/public/AMS/*/orig/FLAIR_mask.nii.gz'}

test = {'img': '/ocean/projects/cis220078p/zli11/MRIPublic/public/AMS/*2/orig/FLAIR.nii.gz',
       'mask': '/ocean/projects/cis220078p/zli11/MRIPublic/public/AMS/*2/orig/FLAIR_mask.nii.gz'}

flair_trainset, flair_testset = load_train_val(train, test, [256, 256], 60)
train_loader = DataLoader(flair_trainset, batch_size = 32, shuffle=True)
test_loader = DataLoader(flair_testset, batch_size = 32, shuffle=True)

/ocean/projects/cis220078p/zli11/MRIPublic/public/AMS/144/orig/FLAIR.nii.gz
/ocean/projects/cis220078p/zli11/MRIPublic/public/AMS/144/orig/FLAIR_mask.nii.gz


ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 

In [ ]:
def train(model, epoch, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0
    IOU = 0
    batch_bar = tqdm.tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5)
    for i, data in enumerate(dataloader, 0):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images.float())
        loss = criterion(outputs, labels.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        jaccard = JaccardIndex('multiclass',num_classes=2).to(device)
        acc = jaccard(outputs, labels.int())
        
        acc = torch.Tensor.cpu(acc)
        IOU += acc
        
        batch_bar.set_postfix(
            acc='{:.04f}%'.format(100*IOU/(i+1)),
            loss='{:.04f}'.format(float(running_loss)/(i+1)),
            lr='{:.04f}'.format(float(optimizer.param_groups[0]['lr'])))
        batch_bar.update()
    batch_bar.close()

    epoch_acc = IOU/len(dataloader)
    epoch_loss = float(running_loss/len(dataloader))
    

    return epoch_acc * 100, epoch_loss

def validate(model, dataloader, criterion):
    model.eval()
    batch_bar = tqdm.tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    IOU = 0.0
    running_loss = 0.0

    for i, data in enumerate(dataloader):
        
        # Move images to device
        images, labels = data[0].to(device), data[1].to(device)
        
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images.float())
            loss = criterion(outputs, labels)
        
        jaccard = JaccardIndex('multiclass',num_classes=2).to(device)
        acc = jaccard(outputs, labels.int())
        acc = torch.Tensor.cpu(acc)
        IOU += acc
        running_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * IOU / (i + 1)),
            loss="{:.04f}".format(float(running_loss / (i + 1))))

        batch_bar.update()
        
        if i % 9 == 0:
            test_input = images[0,:,:,:].expand(1,-1, -1, -1) 
            test_output = model(test_input.float())
            test_output = torch.Tensor.cpu(test_output)
            test_output = test_output.detach().numpy()
            test_input = torch.Tensor.cpu(test_input)
            label_img = torch.Tensor.cpu(labels[0,:,:,:])
            label_img = label_img.detach().numpy()



            plt.figure(1)
            f, axs = plt.subplots(2,2,figsize=(15,15))
            plt.subplot(141)
            plt.imshow(test_output[0,0,:,:], cmap = 'gray')
            plt.subplot(142)
            plt.imshow(test_input[0,0,:,:], cmap = 'gray')
            plt.subplot(143)
            plt.imshow(test_input[0,0,:,:], cmap = 'gray')
            plt.imshow(test_output[0,0,:,:], cmap = 'gray', alpha = 0.5)
            plt.subplot(144)
            plt.imshow(test_input[0,0,:,:], cmap = 'gray')
            plt.imshow(label_img[0,:,:], cmap = 'gray', alpha = 0.5)

            plt.show()

        
    batch_bar.close()
    
    epoch_acc = IOU/len(dataloader)
    epoch_loss = float(running_loss/len(dataloader))

    return epoch_acc * 100, epoch_loss


In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
best_valacc = 0.0

for epoch in range(config['epochs']):
    
    # if epoch %49 == 0:
    #     optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], betas=(0.9, 0.999), eps=1e-08)
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.8, patience=2)
    #     curr_lr = float(optimizer.param_groups[0]['lr'])
    #     print(f'Epoch: {epoch}, Learning rate: {curr_lr}')

    epoch_train_acc, epoch_train_loss = train(model, epoch, train_loader, optimizer, criterion)
    curr_lr = float(optimizer.param_groups[0]['lr'])
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        config['epochs'],
        epoch_train_acc,
        epoch_train_loss,
        curr_lr))
    epoch_test_acc, epoch_test_loss = validate(model, test_loader, criterion)
    scheduler.step(epoch_test_loss)
    
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(epoch_test_acc, epoch_test_loss))

    
    if epoch_test_acc >= best_valacc:
      #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
        print("Saving model")
        torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  #'scheduler_state_dict':scheduler.state_dict(),
                  'val_acc': epoch_test_acc, 
                  'epoch': epoch}, './7T_No_Norm.pth')
        best_valacc = epoch_test_acc
      # wandb.save('7T_no_Norm.pth')
run.finish()